***GENERATED CODE FOR sqlserver PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Occupation', 'transformation_label': 'String Indexer'}], 'feature': 'Occupation', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '31', 'mean': '', 'stddev': '', 'min': 'Agricultural and food scientists', 'max': 'Urban and regional planners', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Occupation'}, {'feature_label': 'Occupation', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Occupation')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'M_workers', 'transformation_label': 'String Indexer'}], 'feature': 'M_workers', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '31', 'mean': '93.65', 'stddev': '130.92', 'min': '100', 'max': '97', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'M_workers'}, {'feature_label': 'M_workers', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('M_workers')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'M_weekly', 'transformation_label': 'String Indexer'}], 'feature': 'M_weekly', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '31', 'mean': '950.69', 'stddev': '340.39', 'min': '1009', 'max': 'Na', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'M_weekly'}, {'feature_label': 'M_weekly', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('M_weekly')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'F_workers', 'transformation_label': 'String Indexer'}], 'feature': 'F_workers', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '31', 'mean': '81.16', 'stddev': '154.49', 'min': '0', 'max': '9', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'F_workers'}, {'feature_label': 'F_workers', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('F_workers')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'F_weekly', 'transformation_label': 'String Indexer'}], 'feature': 'F_weekly', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '31', 'mean': '730.6', 'stddev': '300.92', 'min': '1215', 'max': 'Na', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'F_weekly'}, {'feature_label': 'F_weekly', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('F_weekly')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'All_workers', 'transformation_label': 'String Indexer'}], 'feature': 'All_workers', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '31', 'mean': '174.87', 'stddev': '252.26', 'min': '114', 'max': '948', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'All_workers'}, {'feature_label': 'All_workers', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('All_workers')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'All_weekly', 'transformation_label': 'String Indexer'}], 'feature': 'All_weekly', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '31', 'mean': '903.63', 'stddev': '313.16', 'min': '1051', 'max': 'Na', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'All_weekly'}, {'feature_label': 'All_weekly', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('All_weekly')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pyspark


def linear_regression(sparkDF, labels, features):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    lr_model = LinearRegression()
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    lr_model.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % lr_model.score(X_test, y_test))

    data = {'model': lr_model,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
#%run sqlserverHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': '172.17.0.6', 'port': '1433', 'password': 'num_sql@1433', 'database': 'numtra', 'optionalDB': 'numtra', 'user': 'SA', 'qtype': 'dbtable', 'query': 'PredictHighestIncomeTrain', 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run sqlserverHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(sqlserver,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Occupation", "transformation_label": "String Indexer"}], "feature": "Occupation", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "31", "mean": "", "stddev": "", "min": "Agricultural and food scientists", "max": "Urban and regional planners", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Occupation"}, {"transformationsData": [{"feature_label": "M_workers", "transformation_label": "String Indexer"}], "feature": "M_workers", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "31", "mean": "93.65", "stddev": "130.92", "min": "100", "max": "97", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "M_workers"}, {"transformationsData": [{"feature_label": "M_weekly", "transformation_label": "String Indexer"}], "feature": "M_weekly", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "31", "mean": "950.69", "stddev": "340.39", "min": "1009", "max": "Na", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "M_weekly"}, {"transformationsData": [{"feature_label": "F_workers", "transformation_label": "String Indexer"}], "feature": "F_workers", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "31", "mean": "81.16", "stddev": "154.49", "min": "0", "max": "9", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "F_workers"}, {"transformationsData": [{"feature_label": "F_weekly", "transformation_label": "String Indexer"}], "feature": "F_weekly", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "31", "mean": "730.6", "stddev": "300.92", "min": "1215", "max": "Na", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "F_weekly"}, {"transformationsData": [{"feature_label": "All_workers", "transformation_label": "String Indexer"}], "feature": "All_workers", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "31", "mean": "174.87", "stddev": "252.26", "min": "114", "max": "948", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "All_workers"}, {"transformationsData": [{"feature_label": "All_weekly", "transformation_label": "String Indexer"}], "feature": "All_weekly", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "31", "mean": "903.63", "stddev": "313.16", "min": "1051", "max": "Na", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "All_weekly"}, {"feature": "Occupation_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "31", "mean": "15.0", "stddev": "9.09", "min": "0.0", "max": "30.0", "missing": "0"}, "updatedLabel": "Occupation_stringindexer_..."}, {"feature": "M_workers_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "31", "mean": "13.13", "stddev": "8.89", "min": "0.0", "max": "28.0", "missing": "0"}, "updatedLabel": "M_workers_stringindexer_t..."}, {"feature": "M_weekly_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "31", "mean": "4.39", "stddev": "5.48", "min": "0.0", "max": "16.0", "missing": "0"}, "updatedLabel": "M_weekly_stringindexer_tr..."}, {"feature": "F_workers_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "31", "mean": "10.0", "stddev": "7.81", "min": "0.0", "max": "24.0", "missing": "0"}, "updatedLabel": "F_workers_stringindexer_t..."}, {"feature": "F_weekly_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "31", "mean": "1.77", "stddev": "3.1", "min": "0.0", "max": "10.0", "missing": "0"}, "updatedLabel": "F_weekly_stringindexer_tr..."}, {"feature": "All_workers_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "31", "mean": "14.03", "stddev": "9.04", "min": "0.0", "max": "29.0", "missing": "0"}, "updatedLabel": "All_workers_stringindexer..."}, {"feature": "All_weekly_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "31", "mean": "6.13", "stddev": "6.6", "min": "0.0", "max": "19.0", "missing": "0"}, "updatedLabel": "All_weekly_stringindexer_..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run sqlserverHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=linear_regression(autofe, ["Occupation_stringindexer", "M_workers_stringindexer", "M_weekly_stringindexer", "F_workers_stringindexer", "F_weekly_stringindexer", "All_weekly_stringindexer"], "All_workers_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
